In [2]:
from azure.planetarycomputer import PlanetaryComputerClient
from azure.identity import DefaultAzureCredential

In [3]:
credential = DefaultAzureCredential()
endpoint = "https://test-accessibility.h5d5a9crhnc8deaz.uksouth.geocatalog.spatio.azure.com"
collection_id = "python-sdk-test-collection"
# MPC Pro Client
client = PlanetaryComputerClient(endpoint=endpoint, credential=DefaultAzureCredential())

In [4]:
collection_payload = {
    "description": "Tutorial collection for integration tests",
    "extent": {
        "spatial": {"bbox": [[-180, -90, 180, 90]]},
        "temporal": {"interval": [["2020-01-01T00:00:00Z", None]]}
    },
    "id": collection_id,
    "license": "CC-BY-4.0",
    "links": [],
    "stac_version": "1.0.0",
    "title": "Python SDK Test Collection",
    "type": "Collection",
    "item_assets": {
        "GEC": {
            "type": "image/tiff; application=geotiff; profile=cloud-optimized",
            "roles": ["data"],
            "title": "VV polarization",
            "description": "Gamma naught values corrected for terrain",
            "raster:bands": [{
                "nodata": -32768,
                "data_type": "uint8",
                "spatial_resolution": 0.477
            }]
        }
    }
}

In [ ]:
# Create Collection
# TODO: Fix TypeSpec so that 200 is a valid response code when begin_create_or_update is used to make the notebook idempotent
get_all_collections_response = client.stac_collection_operations.get_all()
print(collections)
if any(c["id"] == collection_id for c in get_all_collections_response["collections"]):
    print(f"Collection {collection_id} already exists.")
else:
    collection_create_operation = client.stac_collection_operations.begin_create(body=collection_payload, polling=False)
    collection_create_operation.result()
 

{'collections': [{'id': 'example-collection', 'type': 'Collection', 'links': [{'rel': 'items', 'type': 'application/geo+json', 'href': 'https://test-accessibility.h5d5a9crhnc8deaz.uksouth.geocatalog.spatio.azure.com/stac/collections/example-collection/items'}, {'rel': 'parent', 'type': 'application/json', 'href': 'https://test-accessibility.h5d5a9crhnc8deaz.uksouth.geocatalog.spatio.azure.com/stac/'}, {'rel': 'root', 'type': 'application/json', 'href': 'https://test-accessibility.h5d5a9crhnc8deaz.uksouth.geocatalog.spatio.azure.com/stac/'}, {'rel': 'self', 'type': 'application/json', 'href': 'https://test-accessibility.h5d5a9crhnc8deaz.uksouth.geocatalog.spatio.azure.com/stac/collections/example-collection'}], 'title': 'Example Collection', 'extent': {'spatial': {'bbox': [[-180, -90, 180, 90]]}, 'temporal': {'interval': [['2018-01-01T00:00:00Z', '2018-12-31T23:59:59Z']]}}, 'license': 'CC-BY-4.0', 'description': 'An example collection', 'stac_version': '1.0.0', 'msft:_created': '2025-03

In [18]:
# Get Collection
client.stac_collection_operations.get(collection_id=collection_id)

{'id': 'python-sdk-test-collection', 'type': 'Collection', 'links': [{'rel': 'items', 'type': 'application/geo+json', 'href': 'https://test-accessibility.h5d5a9crhnc8deaz.uksouth.geocatalog.spatio.azure.com/stac/collections/python-sdk-test-collection/items'}, {'rel': 'parent', 'type': 'application/json', 'href': 'https://test-accessibility.h5d5a9crhnc8deaz.uksouth.geocatalog.spatio.azure.com/stac/'}, {'rel': 'root', 'type': 'application/json', 'href': 'https://test-accessibility.h5d5a9crhnc8deaz.uksouth.geocatalog.spatio.azure.com/stac/'}, {'rel': 'self', 'type': 'application/json', 'href': 'https://test-accessibility.h5d5a9crhnc8deaz.uksouth.geocatalog.spatio.azure.com/stac/collections/python-sdk-test-collection'}], 'title': 'Python SDK Test Collection', 'extent': {'spatial': {'bbox': [[-180, -90, 180, 90]]}, 'temporal': {'interval': [['2020-01-01T00:00:00Z', None]]}}, 'license': 'CC-BY-4.0', 'description': 'Tutorial collection for integration tests', 'item_assets': {'GEC': {'type': '

In [19]:
import json
from io import BytesIO
import requests

thumbnail_url = "https://ai4edatasetspublicassets.blob.core.windows.net/assets/pc_thumbnails/sentinel-2.png"
# Define thumbnail asset metadata
data_str = json.dumps({
    "key": "thumbnail",
    "href": thumbnail_url,
    "type": "image/png",
    "roles": ["thumbnail"],
    "title": "Sentinel-2 preview"
})

# Download thumbnail
thumbnail_bytes = BytesIO(requests.get(thumbnail_url).content)
thumbnail_tuple = ("thumbnail.png", thumbnail_bytes)

In [20]:
# Create Collection Asset
client.stac_collection_assets.create(
    collection_id=collection_id, 
    body={
        "data": json.loads(data_str),
        "file": thumbnail_tuple
    })

{'type': 'Collection', 'id': 'python-sdk-test-collection', 'stac_version': '1.0.0', 'description': 'Tutorial collection for integration tests', 'links': [{'rel': 'items', 'type': 'application/geo+json', 'href': 'https://test-accessibility.h5d5a9crhnc8deaz.uksouth.geocatalog.spatio.azure.com/stac/collections/python-sdk-test-collection/items'}, {'rel': 'parent', 'type': 'application/json', 'href': 'https://test-accessibility.h5d5a9crhnc8deaz.uksouth.geocatalog.spatio.azure.com/stac/'}, {'rel': 'root', 'type': 'application/json', 'href': 'https://test-accessibility.h5d5a9crhnc8deaz.uksouth.geocatalog.spatio.azure.com/stac/'}, {'rel': 'self', 'type': 'application/json', 'href': 'https://test-accessibility.h5d5a9crhnc8deaz.uksouth.geocatalog.spatio.azure.com/stac/collections/python-sdk-test-collection'}], 'item_assets': {'GEC': {'type': 'image/tiff; application=geotiff; profile=cloud-optimized', 'roles': ['data'], 'title': 'VV polarization', 'description': 'Gamma naught values corrected for

In [21]:
search_body = {
    "collections": [collection_id],
    "bbox": [-70.9, -33.5, -70.7, -33.3],
    "datetime": "2023-01-01T00:00:00Z/2023-12-31T23:59:59Z"
}
# Create Search
search_response = client.stac_search_operations.create(body=search_body)

In [22]:
search_response

{'type': 'FeatureCollection', 'features': [], 'links': [{'rel': 'root', 'type': 'application/json', 'href': 'https://test-accessibility.h5d5a9crhnc8deaz.uksouth.geocatalog.spatio.azure.com/stac/'}, {'rel': 'self', 'type': 'application/json', 'href': 'https://test-accessibility.h5d5a9crhnc8deaz.uksouth.geocatalog.spatio.azure.com/stac/search?api-version=2025-04-30-preview'}]}

In [53]:
# Delete ingestion sources

list_ingestion_sources_response = client.ingestion_sources.list_all()
print(list_ingestion_sources_response["value"])

for ingestion_source in list_ingestion_sources_response["value"]:
    client.ingestion_sources.delete(id=ingestion_source["id"])

list_ingestion_sources_response = client.ingestion_sources.list_all()
print(list_ingestion_sources_response["value"])

if not list_ingestion_sources_response["value"]:
    print("No ingestion sources found, as expected.")

[{'id': '990ed5b4-c7fa-4544-8253-9904f47f2317', 'kind': 'BlobManagedIdentity', 'created': '2025-08-04T21:33:40.7595453Z'}]
[]
No ingestion sources found, as expected.


In [ ]:
create_ingestion_source_response = client.ingestion_sources.create(
    ingestion_source= {
        "connectionInfo": {
                "containerUrl": "https://datazoo.blob.core.windows.net/sentinel2static",
                "objectId": "ebad594e-84af-49da-89db-7bffc9c39f3a"
            },
        "kind": "BlobManagedIdentity"
    }
)

print(create_ingestion_source_response)
ingestion_source_id

{'id': 'b34de676-8a4f-473d-b8a7-59970a411e23', 'kind': 'BlobManagedIdentity', 'created': '2025-08-04T21:37:38.4058314Z', 'connectionInfo': {'containerUrl': 'https://datazoo.blob.core.windows.net/sentinel2static', 'objectId': 'ebad594e-84af-49da-89db-7bffc9c39f3a'}}


In [65]:
ingestion_source_id

'b34de676-8a4f-473d-b8a7-59970a411e23'

In [66]:
client.ingestion_sources.get(ingestion_source_id)

{'id': 'b34de676-8a4f-473d-b8a7-59970a411e23', 'kind': 'BlobManagedIdentity', 'created': '2025-08-04T21:37:38.4058314Z', 'connectionInfo': {'containerUrl': 'https://datazoo.blob.core.windows.net/sentinel2static', 'objectId': 'ebad594e-84af-49da-89db-7bffc9c39f3a'}}

In [82]:
# create ingestion
response = client.ingestions.create(
        collection_id=collection_id,
        definition={
            "name": "Sentinel-2 CAMSFO Ingestion",
            "importType": "StaticCatalog",
            "keepOriginalAssets": False,
            "skipExistingItems": False,
            "sourceCatalogUrl": "https://datazoo.blob.core.windows.net/sentinel2/S2A_MSIL2A_20230815T030551_N0509_R075_T50TMK_20230815T082905.SAFE/GRANULE/L2A_T50TMK_A042543_20230815T031445/AUX_DATA/AUX_CAMSFO",
        },
    )

In [83]:
response
ingestion_id = response["id"]

In [84]:
# create run
ingestion_id = ingestion_id
client.ingestions.ingestion_runs.create(
    collection_id = collection_id,
    ingestion_id = response.id
)

{'id': '1b8eee37-6926-4c55-bbfc-30c47ce0f599', 'creationTime': '2025-08-04T21:43:30.3463438Z', 'sourceCatalogUrl': 'https://datazoo.blob.core.windows.net/sentinel2/S2A_MSIL2A_20230815T030551_N0509_R075_T50TMK_20230815T082905.SAFE/GRANULE/L2A_T50TMK_A042543_20230815T031445/AUX_DATA/AUX_CAMSFO', 'skipExistingItems': False, 'keepOriginalAssets': False, 'operation': {'id': '1b8eee37-6926-4c55-bbfc-30c47ce0f599', 'status': 'Pending', 'statusHistory': [{'status': 'Pending', 'timestamp': '2025-08-04T21:43:30.3195115Z'}], 'creationTime': '2025-08-04T21:43:30.3195097Z', 'totalItems': 0, 'totalPendingItems': 0, 'totalSuccessfulItems': 0, 'totalFailedItems': 0}}

In [85]:
# list runs
client.ingestions.ingestion_runs.list_all(
         ingestion_id=ingestion_id,
        collection_id=collection_id,
    )

{'value': [{'id': '1b8eee37-6926-4c55-bbfc-30c47ce0f599', 'creationTime': '2025-08-04T21:43:30.3463438Z', 'sourceCatalogUrl': 'https://datazoo.blob.core.windows.net/sentinel2/S2A_MSIL2A_20230815T030551_N0509_R075_T50TMK_20230815T082905.SAFE/GRANULE/L2A_T50TMK_A042543_20230815T031445/AUX_DATA/AUX_CAMSFO', 'skipExistingItems': False, 'keepOriginalAssets': False, 'operation': {'id': '1b8eee37-6926-4c55-bbfc-30c47ce0f599', 'status': 'Failed', 'statusHistory': [{'status': 'Pending', 'timestamp': '2025-08-04T21:43:30.319509Z'}, {'status': 'Running', 'timestamp': '2025-08-04T21:43:30.81894Z'}, {'status': 'Failed', 'timestamp': '2025-08-04T21:43:31.191037Z', 'errorCode': 'PublicAccessRestricted', 'errorMessage': 'Error retrieving content from public source. Reason: Public access is not permitted on this storage account. Source: https://datazoo.blob.core.windows.net/sentinel2/S2A_MSIL2A_20230815T030551_N0509_R075_T50TMK_20230815T082905.SAFE/GRANULE/L2A_T50TMK_A042543_20230815T031445/AUX_DATA/AU

In [86]:
client.ingestions.begin_delete(
    collection_id = collection_id,
    ingestion_id = response.id,
    polling = False
)

In [ ]:
geometry = {'type': 'Polygon',
 'coordinates': [[[-22.3257229, 63.0320389],
   [-22.4023885, 63.047747],
   [-22.4016604, 63.0485574],
   [-22.4028808, 63.0487921],
   [-22.4028386, 63.0488391],
   [-22.4054314, 63.049339],
   [-22.4050627, 63.04975],
   [-22.8228334, 63.1299748],
   [-22.822979, 63.12981],
   [-22.8231846, 63.1298495],
   [-22.8240444, 63.1288771],
   [-22.8563547, 63.1348829],
   [-22.9863784, 63.1590603],
   [-23.0464985, 64.0127397],
   [-20.8002655, 64.0270166],
   [-20.8070398, 63.0416105],
   [-22.3257229, 63.0320389]]]}

bbox = [-23.04649848, 63.03203887, -20.80026553, 64.02701664]

properties = {'datetime': '2024-02-11T13:12:51.024000Z',
 'platform': 'Sentinel-2A',
 'proj:epsg': 32627,
 'instruments': ['msi'],
 's2:mgrs_tile': '27VVL',
 'constellation': 'Sentinel 2',
 's2:granule_id': 'S2A_OPER_MSI_L2A_TL_MSFT_20240211T171119_A045123_T27VVL_N05.10',
 'eo:cloud_cover': 82.178414,
 's2:datatake_id': 'GS2A_20240211T131251_045123_N05.10',
 's2:product_uri': 'S2A_MSIL2A_20240211T131251_N0510_R081_T27VVL_20240211T171119.SAFE',
 's2:datastrip_id': 'S2A_OPER_MSI_L2A_DS_MSFT_20240211T171119_S20240211T131252_N05.10',
 's2:product_type': 'S2MSI2A',
 'sat:orbit_state': 'descending',
 's2:datatake_type': 'INS-NOBS',
 's2:generation_time': '2024-02-11T17:11:19.61018Z',
 'sat:relative_orbit': 81,
 's2:water_percentage': 11.673754,
 's2:mean_solar_zenith': 77.7957267607447,
 's2:mean_solar_azimuth': 173.129930934397,
 's2:processing_baseline': '05.10',
 's2:snow_ice_percentage': 5.765103,
 's2:vegetation_percentage': 0.000366,
 's2:thin_cirrus_percentage': 32.264775,
 's2:cloud_shadow_percentage': 0.193934,
 's2:nodata_pixel_percentage': 1.963673,
 's2:unclassified_percentage': 3.7e-05,
 's2:dark_features_percentage': 0.151322,
 's2:not_vegetated_percentage': 0.037065,
 's2:degraded_msi_data_percentage': 0.0235,
 's2:high_proba_clouds_percentage': 26.63025,
 's2:reflectance_conversion_factor': 1.02824644491128,
 's2:medium_proba_clouds_percentage': 23.283391,
 's2:saturated_defective_pixel_percentage': 0.0}

from azure.planetarycomputer.models import StacItem
from azure.planetarycomputer.models import Link
stac_item = StacItem(
    geometry=geometry,
    bbox=bbox,
    stac_version="1.0.0",
    properties=properties,
    links=[Link(rel="collection",
                href="https://test-accessibility.h5d5a9crhnc8deaz.uksouth.geocatalog.spatio.azure.com/stac/collections/python-sdk-test-collection")],
    assets={"AOT": {
            "href": "https://datazoo.blob.core.windows.net/sentinel2static/items/S2A_MSIL2A_20230815T030551_R075_T50TMK_20230815T082905.json"
        }},
    id="S2A_MSIL2A_20230815T030551_R075_T50TMK_20230815T082908",
    collection=collection_id
    )

TypeError: azure.planetarycomputer.models._models.StacItemOrItemCollection.__init__() got multiple values for keyword argument 'type'

In [ ]:
# Create Stac Item
operation = client.stac_items.begin_create(
    collection_id=collection_id,
    body=stac_item
)

In [ ]:
operation.result() # STAC item creation fails

In [ ]:
# List All ingestion Operations
operations = client.ingestion_operations.list_all(collection_id=collection_id)

In [ ]:
len(operations["value"])

In [ ]:
# Get Collection
client.stac_collection_operations.get(collection_id=collection_id)

In [ ]:
render_option = {
    "id": "vv-polarization",
    "name": "VV polarization",
    "description": "VV asset scaled to 0–255 grayscale",
    "type": "raster-tile",
    "options": "assets=GEC&rescale=0,255&colormap_name=gray",
    "minZoom": 8,
    "conditions": [{"property": "sar:polarizations", "value": ["VV"]}]
}

# Create Render Option
client.stac_collection_render_options.create(
    collection_id=collection_id,
    body=render_option
)

In [ ]:
mosaics = {"id": "default1", "name": "Default Mosaic", "description": "", "cql": []}

# Create Mosaic
client.stac_collection_mosaics.add(
    collection_id=collection_id,
    body=mosaics
)

In [3]:
# Delete Collection
collection_delete_operation = client.stac_collection_operations.begin_delete(
    collection_id=collection_id
)

In [ ]:
collection_delete_operation.result()